In [ ]:
# In case of import requirements
# verify version
!python3 --version
!pip install --upgrade luxai_s2
!pip install importlib-metadata==4.13.0
!pip install --upgrade moviepy # needed to render videos of episodes

In [8]:
from luxai_s2.env import LuxAI_S2
from lux.kit import obs_to_game_state, GameState, EnvConfig
from lux.utils import my_turn_to_place_factory, direction_to 
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import random
import math
import cv2

In [ ]:
# Create excel spreadsheets to look at strategies for when to start producing lichen to find weather its more optimal to
#      continually grow lichen, or to hoard water until a certain turn (depending on water gathered?) and mass produce lichen then
# [X] Made graphs and have found good enough growth estimates with tables for further estimation if necessary.
# TODO Update files in repo, or move agent files to new repo. Basically move from beta to season 2.
# [X]
# TODO Clean up structure of notebook cells and segments of the algorithm to make it easier to find where what desicion is being made
# [X]
# TODO Make the agent place a factory right next to one ice tile
# [X]
# TODO Make the agent update its possible placing locations by removing coordinates around placed (friendly and foe) factory tiles
# [X]
# TODO Update the assignments to have n heavy bots on ice gathering just outside the factory, and some light for ore collection
# [X]
# TODO Observe the total ammount of water possible to gather with 1 heavy bot
# [X]
# TODO Fix ordering system so that heavies are assigned to ice tiles right by the factory for instant dropof and charging,
#      and so that the bots are ordered to correct coordinates,
#      and so that the return to factory scheduling works.
# [X]
# TODO Fix bug where only 4 out of 5 factories are built 
# [X]
# TODO Fix placement of factories? So that all place near ice
# [X]
# TODO Fix bug where not every factory is watering lichen
# [X]
# TODO Optimize heavy bots ice collection, by increasing storage limit, and or tweaking their power level threshold
# [X]
# TODO Take second look at spawn selection functions as this might be running into "overtime" making me miss out on placing some crucial factories
#      This can be reduced by looking at efficiency gains that can be made in the startup section
# [X]
# TODO Take second look at border coordinate function to fix it so that it removes first and second rows on all 4 sides
# [X]
# TODO, make selection of spawn points less random, by sorting for rubble value, and selecting first in list, not random
# [X]
# TODO Check if there is logic in heavy and light bots that can be merged to reduce complexity in logic? If not, make sure google sheet matricees
#      are up to date
# [X]
# TODO Doing nothing equals recharging. Rewrite logic for light bots so that they do nothing instead of queueing 6 turn imobilizing recharge action.
# [X]
# TODO Heavy bot sometimes dont have enough power to move back to factory if there is one tile between ice tile being mined and factory
# [X]
# TODO Incorporate a sorting function for rubble tiles as well as proximity to the light bot (changed to factory tile with closest proximity to unit),
#      this allows "bootstrapping" from start pos in factory
# [X]
# TODO Make function that gets the coordinates around factories with only rubble on, and assigns these tiles (by filtering for proximity) to light bots
# [X]
# TODO Take second look at non colision function. Also make ice tiles "no go" for lights.
#      Function should be rewritten so that bot logic places "tile bookings" for where they want to go. 
#      If the booking is in the array of existing bookings, check opposite axis directions if in "tile bookings".
#      If these are not in the np array, select that direction and return the direction.
#      If doing nothing (recharging), place a booking for their current tile
#
#      This booking system should consist of a tuple of two np arrays, that indicate a vector for that unit.
#      Start function by checking straight directions from bot pos, if not occupied, do final check if tile is in "tile bookings" array.
#      This way diagonal directions can be accounted for, for friendly bots
# [X]
# TODO Add function that checks surrounding tiles for hostiles if on ice or factory tile, that affects behaviour of heavy bots (moves bot on hostile heavy bot on ajacent tile, ambush this bot by moving onto this tile)
# [X]
# TODO Develop formula for when to start watering to optimize so that factory water, and a variable (average water gathering from 1 heavy),
#      find the optimal turn for beginning watering
# [X]
# TODO Make sure coords round factory tiles (2 layers) are clear of rubble
# [X]
# TODO Update spawn selection so that spawn rubble values are set to average rubble value for the 9 factory tiles and one tile around these.
#      This allows selecting areas with lower rubble
# [X]
# TODO Make heavy bots more defensive, and move to factory tile if low on power (made overwatch decision more random for moving home)
# [X]


"""HIGH PRIORITY"""
# TODO Change spawn selection from using coordinates to use Djikstra algorithm to calculate distances from each ice tile, which then returns the
#      average of the heatmap instead of ajacent rubble tiles for that ice tile.
#      This means retooling the spawn filtering section to evaluate the spawns by their "distance score".
# []

# TODO Remove enemy factory tile coordinates from closest rubble low list to avoid clearing out rubble for opponent
# []

# TODO Add "hostile_facotry_ajacent" to heavy and light overwatch functions and checks to avoid random choice of trying to attack, when it should
#      move home (always when next to hostile factory)
# []


"""MEDIUM PRIORITY"""
# TODO Introduce switch for light bots toggling between curren behaviour (after turn 200), and creation of a "human chain" on a ore tile, if 
#      there is one ore tile N distance away from the factory center tile (can find where center coords are defined).
#      Here one light bot is assigned as miner, while rest follows, and charges during daytime, and transfers M power forwards to mining bot.
#      The mining bot transfers ore towards factory tile, and supporting bots towards factory, if bot is on coordinate towards factory.
#      If no bot can recieve, the light bot starts moving towards factory to drop off ore (it has ore greater than free cargo space)
# []
# TODO Make bots with ore assignments fetch power from factory with "pickup", as power is not utilized to full extent
# []


"""LOW PRIORITY"""
# TODO Check if bidding 1 water and metal affects the number of factories that can be placed, if factories dont cost metal, bid 1 as default. This
#      secures the first, best location more often. Combined with selection of 3rd best spot forces opponent to choose more bad spots with more rubble
# []
# TODO Make toggle between construction of heavy and light bots based on real env turns

# TODO Evaluate weather to use 150-N turns to gather ore for multiplication of bots before water hoarding
# []


In [24]:
env = LuxAI_S2() # create the environment object
obs = env.reset(seed=361203580) # resets an environment with a seed

# the observation is always composed of observations for both players.
obs.keys(), obs["player_0"].keys()

# visualize the environment so far with rgb_array to get a quick look at the map
# dark orange - high rubble, light orange - low rubble
# blue = ice, yellow = ore
img = env.render("rgb_array", width=48, height=48)
px.imshow(img).show()

In [25]:
def rubble_tile_vision(x, y, rubble_map):
    height = len(rubble_map) - 1
    width = len(rubble_map[0]) - 1
    vision_coords =  [

        [x-1, y], [x, y-1], # left and top
        [x+1, y], [x, y+1], # right and bottom
        [x-1, y-1], [x+1, y-1], # diagonals
        [x-1, y+1], [x+1, y+1], # diagonals
        [x-2, y],   [x, y-2], # left and top 
        [x+2, y],   [x, y+2], # right and bottom 
        [x-2, y-1], [x-1, y-2], [x-2, y-2], # left and top diagonals 
        [x+2, y+1], [x+1, y+2], [x+2, y+2], # right and bottom diagonals 
        [x+2, y-1], [x+1, y-2], [x+2, y-2], # right and top diagonals 
        [x-2, y+1], [x-1, y+2], [x-2, y+2], # left and bottom diagonals 

        # feeler coords
        [x-3, y],   [x, y-3], # left and top 
        [x+3, y],   [x, y+3], # right and bottom 

        [x, y-5],
        [x-2, y-3], [x+2, y-3],
        [x-5, y-2], [x-3, y-2], [x+3, y-2], [x+5, y-2],
        [x+4, y-1], [x-4, y-1],
        [x-6, y], [x+6, y],
        [x+4, y+1], [x-4, y+1],
        [x-5, y+2], [x-3, y+2], [x+3, y+2], [x+5, y+2],
        [x-2, y+3], [x+2, y+3],
        [x, y+5],

    ]

    return [(rubble_coord, rubble_map[rubble_coord[0]][rubble_coord[1]]) for rubble_coord in vision_coords 
        if rubble_coord[0] >= 0 and rubble_coord[0] <= width and
           rubble_coord[1] >= 0 and rubble_coord[1] <= height
    ]
'''
                  [ ]
            [ ]   ^-5   [ ]
    [ ]  [ ]      [ ]      [ ]   [ ]
               [ ][ ][ ]   
      [ ]   [ ][ ][ ][ ][ ]   [ ]
[ ]      [ ][ ][ ] x [ ][ ][ ]      [ ]
^-6   [ ]   [ ][ ][ ][ ][ ]   [ ]   ^+6
               [ ][ ][ ]   
    [ ]  [ ]      [ ]      [ ]   [ ]
            [ ]         [ ]
                  [ ]
                  ^+5
'''


def get_factory_tiles_from_center(x, y):
    return np.array([
        (x, y), # factory center
        (x-1, y), (x, y-1), # left and top
        (x+1, y), (x, y+1), # right and bottom
        (x-1, y-1), (x+1, y-1), # diagonals
        (x-1, y+1), (x+1, y+1), # diagonals
    ]) 
'''
[.][.][.]
[.][.][.]
[.][.][.]
'''


def get_factory_occupied_tiles(x, y):
    return np.array([
        (x, y), # factory center
        (x-1, y), (x, y-1), # left and top
        (x+1, y), (x, y+1), # right and bottom
        (x-1, y-1), (x+1, y-1), # diagonals
        (x-1, y+1), (x+1, y+1), # diagonals
        (x-2, y), (x, y-2), # left and top 
        (x+2, y), (x, y+2), # right and bottom 
        (x-2, y-1), (x-1, y-2), (x-2, y-2), # left and top diagonals 
        (x+2, y+1), (x+1, y+2), (x+2, y+2), # right and bottom diagonals 
        (x+2, y-1), (x+1, y-2), (x+2, y-2), # right and top diagonals 
        (x-2, y+1), (x-1, y+2), (x-2, y+2), # left and bottom diagonals 
        (x-3, y), (x, y-3), (x+3, y), (x, y+3), # left and top
        (x-3, y), (x, y-3), (x+3, y), (x, y+3), # right and bottom
        (x-3, y-1), (x-1, y-3), (x-3, y-3), # left and top diagonals 
        (x+3, y+1), (x+1, y+3), (x+3, y+3), # right and bottom diagonals 
        (x+3, y-1), (x+1, y-3), (x+3, y-3), # left and top diagonals 
        (x-3, y+1), (x-1, y+3), (x-3, y+3), # right and bottom diagonals 
        (x-4, y), (x, y-4), (x+4, y), (x, y+4), # left and top
        (x-5, y), (x, y-5), (x+5, y), (x, y+5), # right and bottom
    ]) 
'''
    [ ][ ][.][ ][ ]
    [ ][ ][.][ ][ ]
    [ ][.][.][.][ ]
 [.][.][.][X][.][.][.]
    [ ][.][.][.][ ]
    [ ][ ][.][ ][ ]
    [ ][ ][.][ ][ ]
'''


def get_tiles_by_interval(x, y, interval_x, interval_y):
    coordinates = np.array(
        [(x + n, y + m) for n in range((-1 * interval_x), interval_x +1) for m in range((-1 * interval_y), interval_y +1) if (0 <= (x + n) <= 47) and (0 <= (y + m) <= 47)]
    )
    return coordinates
'''
    ^ interval y
    [ ][ ][ ]
    [ ][ ][ ]
    [ ][ ][ ]
    interval x ->
'''


def get_surrounding_tiles_by_interval(x, y, interval_x, interval_y):
    coordinates = np.array(
        [
            (x + n, y + m) for n in range((-1 * interval_x), interval_x +1) for m in range((-1 * interval_y), interval_y +1) if
            (0 <= (x + n) <= 47) and (n < -1 or n > 1) and
            (0 <= (y + m) <= 47) and (m < -1 or m > 1)
        ]
    )
    return coordinates 
'''
X = not marked factory tiles
    ^ interval y
    [ ][ ][ ][ ][ ][ ][ ]
    [ ][ ][ ][ ][ ][ ][ ]
    [ ][ ][X][X][X][ ][ ]
    [ ][ ][X][X][X][ ][ ]
    [ ][ ][X][X][X][ ][ ]
    [ ][ ][ ][ ][ ][ ][ ]
    [ ][ ][ ][ ][ ][ ][ ]
    interval x ->
'''


# TODO finish this so i can remove opponent factory tile neighbours from rubble removal
def get_factory_neighbouring_tiles(x, y):
    test_generator_coords = np.array([(x-j, y-k) for j in range(2, 3) for k in range(2, 3)])
    coords =  np.array([
        (x-2, y), (x, y-2), # left and top
        (x+2, y), (x, y+2), # right and bottom
        (x-2, y-2), (x+2, y-2), # top left corner
        (x-2, y-2), (x+2, y-2), # diagonals

        (x-2, y+2), (x+2, y+2), # diagonals
        (x-2, y+2), (x+2, y+2), # diagonals
    ])
    return [coord for coord in coords if (coord[0] >= 0 and coord[0] <= 47) and (coord[1] >= 0 and coord[1] <= 47)]
'''
    [.][.][.][.][.][.][.]
    [.][.][.][.][.][.][.]
    [.][.][X][X][X][.][.]
    [.][.][X][X][X][.][.]
    [.][.][X][X][X][.][.]
    [.][.][.][.][.][.][.]
    [.][.][.][.][.][.][.]
'''

def get_bordering_coords(x, y):
    return np.array([
        (x, y), # center
        (x-1, y), (x, y-1), # left and top
        (x+1, y), (x, y+1), # right and bottom
        (x-1, y-1), (x+1, y-1), # diagonals
        (x-1, y+1), (x+1, y+1), # diagonals
    ])


# function used by Archimedes for locating resource border coordinates
def neighbors(x, y):
    return np.array([
        # commented to give buffer, avoiding placing factory on resource
        # layer 1
        #(x-1, y), (x, y-1), # left and top
        #(x+1, y), (x, y+1), # right and bottom

        # layer 2
        (x-2, y), (x, y-2), # left and top 
        (x+2, y), (x, y+2), # right and bottom 

        (x-2, y-1), (x-1, y-2), # (x-2, y-2), # left and top diagonals 
        (x+2, y+1), (x+1, y+2), # (x+2, y+2), # right and bottom diagonals 

        (x+2, y-1), (x+1, y-2), # (x+2, y-2), # right and top diagonals 
        (x-2, y+1), (x-1, y+2), # (x-2, y+2), # left and bottom diagonals 

        # commented away to try to get factory right next to resource
        # layer 3
        #(x-3, y), (x, y-3), (x+3, y), (x, y+3), # left and top
        #(x-3, y), (x, y-3), (x+3, y), (x, y+3), # right and bottom

        #(x-3, y-1), (x-1, y-3), (x-3, y-3), # left and top diagonals 
        #(x+3, y+1), (x+1, y+3), (x+3, y+3), # right and bottom diagonals 

        #(x+3, y-1), (x+1, y-3), (x+3, y-3), # left and top diagonals 
        #(x-3, y+1), (x-1, y+3), (x-3, y+3), # right and bottom diagonals 

    ])
'''
[ ][.][.][.][ ]
[.][ ][ ][ ][.]
[.][ ][X][ ][.]
[.][ ][ ][ ][.]
[ ][.][.][.][ ]
'''


def coord_from_direction(x, y, direction):
    if direction == 0:
        return (x, y)
    elif direction == 1:
        return (x, y-1)
    elif direction == 2:
        return (x+1, y)
    elif direction == 3:
        return (x, y+1)
    elif direction == 4:
        return (x-1, y)
'''
# a[1] = direction (0 = center, 1 = up, 2 = right, 3 = down, 4 = left)
move_deltas = np.array([[0, 0], [0, -1], [1, 0], [0, 1], [-1, 0]])
'''


# function used by Archimedes to check if tile is occupied by other bots
def check_tile_occupation(game_state, unit_x, unit_y, direction, booked_coords, player, opponent):
    friendly_bots = game_state.units[player]
    hostile_bots = game_state.units[opponent]

    friendly_coords = {(unit.pos[0], unit.pos[1]): unit.unit_type for (unit_id, unit) in friendly_bots.items()}
    hostile_coords = {(unit.pos[0], unit.pos[1]): unit.unit_type for (unit_id, unit) in hostile_bots.items()}

    # a[1] = direction (1 = up, 2 = right, 3 = down, 4 = left)
    # move_deltas = np.array([0, -1], [1, 0], [0, 1], [-1, 0]])
    bot_up = (unit_x, unit_y - 1)
    bot_right = (unit_x + 1, unit_y)
    bot_down = (unit_x, unit_y + 1)
    bot_left = (unit_x - 1, unit_y)


    # checks if no friendly is on direction tile, if none, direction remains, else set to 0
    up_friendly = (bot_up not in friendly_coords and bot_up not in booked_coords) * 1
    right_friendly = (bot_right not in friendly_coords and bot_right not in booked_coords) * 2
    down_friendly = (bot_down not in friendly_coords and bot_down not in booked_coords) * 3
    left_friendly = (bot_left not in friendly_coords and bot_left not in booked_coords) * 4

    # checks if hostile bot is on direction tile, if there is one of type 'LIGHT', direction remains, else set to 0
    up_hostile = True if (bot_up not in hostile_coords) or (bot_up in hostile_coords and hostile_coords[bot_up] == 'LIGHT' ) else False
    right_hostile = True if (bot_right not in hostile_coords) or (bot_right in hostile_coords and hostile_coords[bot_right] == 'LIGHT' ) else False
    down_hostile = True if (bot_down not in hostile_coords) or (bot_down in hostile_coords and hostile_coords[bot_down] == 'LIGHT' ) else False
    left_hostile = True if (bot_left not in hostile_coords) or (bot_left in hostile_coords and hostile_coords[bot_left] == 'LIGHT' ) else False

    '''
    # this is where the hostile attack decision making will be placed, as distinguishing between light/heavy is important for combat
    hostiles = [up_hostile, right_hostile, down_hostile, left_hostile]
    '''
    
    # sets non occupied tile directions
    up_free = up_friendly * up_hostile
    right_free = right_friendly * right_hostile
    down_free = down_friendly * down_hostile
    left_free = left_friendly * left_hostile


    # finds possible directions not occupied by friendly bots and hostile bots
    directions = [up_free, right_free, down_free, left_free]
    if direction in directions:
        return direction
    
    elif sum(directions) > 0:
        alt_directions = [d for d in directions if d > 0]
        return random.choice(alt_directions)
    '''
    # this directs the random movement choice in the opposite axis to direction to keep the bot moving in the general direction
    odd_direction = direction % 2 > 0
    for alt_direction in directions:
        alt_horisontal_directions = [d for d in directions if d > 0 and d % 2 == 0]
        if odd_direction and len(alt_horisontal_directions) > 0:
            return random.choice(alt_horisontal_directions)

        alt_vertical_directions = [d for d in directions if d > 0 and d % 2 == 1]
        if not odd_direction and len(alt_vertical_directions) > 0:
            return random.choice(alt_vertical_directions)
    '''

    '''
      []1
    []2[][]4
      []3
    ''' 

    # if all direction tiles occupied with friendly bots, recharge recommended
    return 0


def heavy_overwatch(unit, unit_x, unit_y, on_factory, closest_factory_tile, game_state, opponent):
    # gets hostile bot coords
    hostile_bots = game_state.units[opponent]
    hostile_coords_heavy = [(unit.pos[0], unit.pos[1]) for (unit_id, unit) in hostile_bots.items() if unit.unit_type == 'HEAVY']

    # coords relative to bot position
    bot_up = (unit_x, unit_y - 1)
    bot_right = (unit_x + 1, unit_y)
    bot_down = (unit_x, unit_y + 1)
    bot_left = (unit_x - 1, unit_y)

    # checks if hostile bot is on directly neighbouring tiles, and is of type HEAVY
    up_hostile = True if (bot_up in hostile_coords_heavy) else False
    right_hostile = True if (bot_right in hostile_coords_heavy) else False
    down_hostile = True if (bot_down in hostile_coords_heavy) else False
    left_hostile = True if (bot_left in hostile_coords_heavy) else False
    hostile_on_flank = (up_hostile or right_hostile or down_hostile or left_hostile)

    # check for heavy hostiles on opposing tiles, if on factory tile move on hostile tile, else move to closest factory tile 
    if on_factory and hostile_on_flank:
        up_target = up_hostile * 1
        right_target = right_hostile * 2
        down_target = down_hostile * 3
        left_target = left_hostile * 4
        directions = [up_target, right_target, down_target, left_target]
        target_dir = [target for target in directions if target != 0]

        return random.choice(target_dir)
    
    # attack if energy is greater than 240, but introduced random choice to disengage and move home
    elif not on_factory and hostile_on_flank:
        if unit.power > 240:
            up_target = up_hostile * 1
            right_target = right_hostile * 2
            down_target = down_hostile * 3
            left_target = left_hostile * 4
            directions = [up_target, right_target, down_target, left_target, direction_to(np.array([unit_x, unit_y]), target=closest_factory_tile)]
            target_dir = [target for target in directions if target != 0]
            return random.choice(target_dir)
        
        else:
            return direction_to(src=np.array([unit_x, unit_y]), target=closest_factory_tile)
    
    else:
        return False


def light_overwatch(unit, unit_x, unit_y, game_state, opponent):
    # gets hostile bot coords
    hostile_bots = game_state.units[opponent]
    hostile_coords_heavy = [(unit.pos[0], unit.pos[1]) for (unit_id, unit) in hostile_bots.items() if unit.unit_type == 'HEAVY']
    hostile_coords_light = [(unit.pos[0], unit.pos[1]) for (unit_id, unit) in hostile_bots.items() if unit.unit_type == 'LIGHT']

    # coords relative to bot position
    bot_up = (unit_x, unit_y - 1)
    bot_right = (unit_x + 1, unit_y)
    bot_down = (unit_x, unit_y + 1)
    bot_left = (unit_x - 1, unit_y)

    # checks if light hostile bot is on directly neighbouring tiles
    up_hostile_light = True if (bot_up in hostile_coords_light) else False
    right_hostile_light = True if (bot_right in hostile_coords_light) else False
    down_hostile_light = True if (bot_down in hostile_coords_light) else False
    left_hostile_light = True if (bot_left in hostile_coords_light) else False
    light_hostile_on_flank = (up_hostile_light or right_hostile_light or down_hostile_light or left_hostile_light)

    # checks if heavy hostile bot is on directly neighbouring tiles
    up_hostile_heavy = True if (bot_up in hostile_coords_heavy) else False
    right_hostile_heavy = True if (bot_right in hostile_coords_heavy) else False
    down_hostile_heavy = True if (bot_down in hostile_coords_heavy) else False
    left_hostile_heavy = True if (bot_left in hostile_coords_heavy) else False
    heavy_hostile_on_flank = (up_hostile_heavy or right_hostile_heavy or down_hostile_heavy or left_hostile_heavy)


    # if only heavy hostile on flank, returns direction that is away from heavy hostile
    if heavy_hostile_on_flank and not light_hostile_on_flank:
        if up_hostile_heavy:
            return 3
        elif right_hostile_heavy:
            return 4
        elif down_hostile_heavy:
            return 1
        elif left_hostile_heavy:
            return 2
    
    # if light hostile on flank, attack
    elif light_hostile_on_flank:
        up_target = up_hostile_light * 1
        right_target = right_hostile_light * 2
        down_target = down_hostile_light * 3
        left_target = left_hostile_light * 4
        directions = [up_target, right_target, down_target, left_target]
        target_dir = [target for target in directions if target != 0]
        return random.choice(target_dir)
    
    else:
        return False

   

class Archimedes_Lever():
    def __init__(self, player: str, env_cfg: EnvConfig) -> None:
        self.player = player
        self.opp_player = "player_1" if self.player == "player_0" else "player_0"
        np.random.seed(0)
        self.env_cfg: EnvConfig = env_cfg

    # Setup of early game
    def early_setup(self, step: int, obs, remainingOverageTime: int = 60):
        if step == 0:
            # bid 0 to not waste resources bidding and declare as the default faction
            # you can bid -n to prefer going second or n to prefer going first in placement
            return dict(faction="AlphaStrike", bid=0)
        else:
            game_state = obs_to_game_state(step, self.env_cfg, obs)
            my_turn_to_evaluate_spawns = my_turn_to_place_factory(game_state.teams[self.player].place_first, step)

            # only processes map on my placement turns
            if my_turn_to_evaluate_spawns:
                ### Using numpy arrays, maps out desirable spawn locations
                # gets border indexes of ore coordinates and ice coordinates
                indOre = np.transpose(np.where(game_state.board.ore > 0))
                indIce = np.transpose(np.where(game_state.board.ice > 0))
                indFactories = np.transpose(np.where(game_state.board.factory_occupancy_map >= 0))

                # finds possible spawns around resources
                array_spawns = np.unique(np.concatenate(
                    [neighbors(*ind) for ind in indIce],
                    #[neighbors(*ind) for ind in iceSpawns],
                    ), axis=0)

                # finds tiles that are occupied or invalid spawn coords
                array_occupied_coords = np.unique(np.concatenate([get_factory_occupied_tiles(*i) for i in indFactories]), axis=0) if len(indFactories) > 0 else []
                array_invalid_coords = np.unique(np.concatenate(
                    (np.concatenate([get_bordering_coords(*ind) for ind in indOre], axis=0),
                    np.concatenate([get_bordering_coords(*ind) for ind in indIce], axis=0))
                ), axis=0)


                # filters spawn coords for outside map coords (and those doesn't include spawns on map border), and those overlapping ivalid or occupied tiles
                x_search_radius = 3
                y_search_radius = 3
                array_inside_map_spawns = [
                    (np.array(
                        [spawn_coord[0], spawn_coord[1]]), # first element in tuple is np array coordinate
                        np.average(                        # second elment in tuple is average rubble value from ajacent coords
                            [game_state.board.rubble[c[0]][c[1]] for c in get_surrounding_tiles_by_interval(x=spawn_coord[0], y=spawn_coord[1], interval_x=x_search_radius, interval_y=y_search_radius)]
                        )
                    ) for spawn_coord in array_spawns if (1 <= spawn_coord[0] <= 46) and (1 <= spawn_coord[1] <= 46)
                ]
                array_valid_spawns = [coord for coord in array_inside_map_spawns if 
                    all(all(coord[0] == inv_coord) == False for inv_coord in array_invalid_coords) and 
                    all(all(coord[0] == occ_coord) == False for occ_coord in array_occupied_coords)
                ]

                '''
                desirable_coordinates_filtered = np.copy(game_state.board.valid_spawns_mask)
                desirable_coordinates_filtered[desirable_coordinates_filtered == 1] = 0
                for coord in array_valid_spawns:
                    desirable_coordinates_filtered[coord[0]][coord[1]] == 1
                # TODO mirror filtering here
                # visualizes the map and AI vision
                img = env.render("rgb_array", width=48, height=48)
                #px.imshow(game_state.board.rubble.T).show()
                px.imshow(img).show()
                px.imshow(desirable_coordinates_filtered.T).show()
                '''


                ### Factory placement period
                # how much water and metal you have in your starting pool to give to new factories
                water_left = game_state.teams[self.player].water
                metal_left = game_state.teams[self.player].metal
                
                # how many factories you have left to place
                factories_to_place = game_state.teams[self.player].factories_to_place
                # whether it is your turn to place a factory
                my_turn_to_place = my_turn_to_place_factory(game_state.teams[self.player].place_first, step)

                if factories_to_place > 0 and my_turn_to_place and len(array_valid_spawns) > 0:
                    # we will spawn our factory in a random location with 150 metal and water if it is our turn to place
                    array_sorted_spawns = sorted(array_valid_spawns, key=lambda coord: coord[1])
                    #print("spawn choices", len(array_sorted_spawns))
                    #print(array_sorted_spawns[0][0])
                    spawn_loc = array_sorted_spawns[0][0]

                    return dict(spawn=spawn_loc, metal=150, water=150)

                else:
                    # returns empty dictionary if there are no  valid spawn points to choose from
                    return dict()

            else:
                # returns empty dictionary if no decisions were reached
                return dict()


    
    # Setup of logic in the act phase
    def act(self, step: int, obs, remainingOverageTime: int = 60):
        # info used to make decisions in the act phase
        actions = dict()
        game_state: GameState = obs_to_game_state(step, self.env_cfg, obs)
        factories = game_state.factories[self.player]
        game_state.teams[self.player].place_first

        # storing of info about factories
        factory_tiles, factory_units, factory_centers = [], [], []
        for unit_id, factory in factories.items():
            center = factory.pos
            tiles = get_factory_tiles_from_center(x=center[0], y=center[1])
            factory_centers += [center]
            for tile in tiles:
                factory_tiles += [tile]
            factory_units += [factory]
        factory_tiles = np.array(factory_tiles)

        ### Creation of bots
        heavy_bot_cost = self.env_cfg.ROBOTS["HEAVY"]
        light_bot_cost = self.env_cfg.ROBOTS["LIGHT"]

        # heavy bots
        for unit_id, factory in factories.items():
            if factory.power >= heavy_bot_cost.POWER_COST and factory.cargo.metal >= heavy_bot_cost.METAL_COST:
                actions[unit_id] = factory.build_heavy()

        
        # light bots
        if game_state.real_env_steps >= 2:
            for unit_id, factory in factories.items():
                if factory.power >= light_bot_cost.POWER_COST and factory.cargo.metal >= light_bot_cost.METAL_COST:
                    actions[unit_id] = factory.build_light()


        

        #print(self.player)
        #print("turn:", game_state.real_env_steps)

        ### Decides when the factories should start growing Lichen
        # Watering lichen if at right turn
        '''
        # starting idea of some sort of regulating mechanism on when to start watering
        # see google sheets for optimal time to start watering by looking at water cargo,
        # number of turns left, 
        # and the water cost from the calculus of water consumption growth
        '''
        growth_turn = 990 - 250 # calculations suggenst 77 turns prior
        turns_left = 988 - game_state.real_env_steps
        grow = game_state.real_env_steps >= growth_turn
        for factory_id, factory in factories.items():
            #print(factory_id, "has;", factory.cargo.metal, "metal", factory.cargo.water, "water", factory.cargo.ice, "ice", factory.power, "power", "watering cost: ", factory.water_cost(game_state))
            water_cost_reactor = turns_left
            water_per_turn = 3.2
            water_formula = (factory.cargo.water - water_cost_reactor + (water_per_turn * turns_left)) > (factory.water_cost(game_state) * turns_left)
            #print(factory_id, factory.cargo.water, factory.water_cost(game_state))
            if game_state.real_env_steps >= growth_turn and water_formula:
                actions[factory_id] = factory.water()
        '''
        # TODO continue this experiment of water to power conversion to see what strategies this unlocks, such as more efficient clearing of rubble,
        #      or mining assignments picking up power for ore mining
        grow = game_state.real_env_steps % 3 == 0
        for factory_id, factory in factories.items():
            #print(factory_id, "has;", factory.cargo.metal, "metal", factory.cargo.water, "water", factory.cargo.ice, "ice", factory.power, "power", "watering cost: ", factory.water_cost(game_state))
            if grow:
                actions[factory_id] = factory.water()
        '''

        ### Finds all ice and ore tiles
        ice_map = game_state.board.ice 
        ore_map = game_state.board.ore 
        rubble_map = game_state.board.rubble
        ice_tile_locations = np.argwhere(ice_map == 1) # numpy magic to get the position of every ice tile
        ore_tile_locations = np.argwhere(ore_map == 1) # numpy magic to get the position of every ore tile
        rubble_tile_locations = np.argwhere(rubble_map > 0) # numpy magic to get the position of every rubble tile
        rubble_tile_locations_low_rubble = np.argwhere(rubble_map <= 20)
        units = game_state.units[self.player]

        ### Specifies assignments for heavy and light bots
        light_ore_miners = 0
        mine_assignments = 0
        rubble_assignments = 0


        # booked tiles for avoiding 
        move_bookings = []

        for unit_id, unit in units.items():
            isHeavy = unit.unit_type == 'HEAVY'
            isLight = unit.unit_type == 'LIGHT'

            # info about closest map tiles
            closest_factory_tile = sorted(factory_tiles, key=lambda p: (p[0] - unit.pos[0])**2 + (p[1] - unit.pos[1])**2 )[0]
            closest_ice_tile = sorted(ice_tile_locations, key=lambda p: (p[0] - unit.pos[0])**2 + (p[1] - unit.pos[1])**2 )[0]
            

            cargo_limit_ice = 960
            cargo_limit_ore = 50
            on_factory = all(unit.pos == closest_factory_tile)
            ajacent_factory = 1 >= abs((unit.pos[0] - closest_factory_tile[0])**2 + (unit.pos[1] - closest_factory_tile[1])**2)


            ### Heavy bot logic
            if isHeavy:
                below_power_threshold_heavy = unit.power <= 100
                below_power_move_heavy = unit.power <= 50
                free_cargo_ice = unit.cargo.ice < cargo_limit_ice
                recharge_need_heavy = math.floor(1000 - unit.power)
                on_ice = all(unit.pos == closest_ice_tile)

                # Print out bot info 
                #print(unit.unit_type, unit_id, "at:", unit.pos, "power and ice:", unit.power, unit.cargo.ice, "unit queue:", len(unit.action_queue))
                
                ### performs overwatch check to see if HEAVY bots on flanks
                overwatch_check = heavy_overwatch(
                    unit=unit,
                    unit_x=unit.pos[0],
                    unit_y=unit.pos[1],
                    on_factory=on_factory,
                    closest_factory_tile=closest_factory_tile,
                    game_state=game_state,
                    opponent=self.opp_player
                )

                # has hostile HEAVY on flank, moves to attack, or to factory tile if not already on one
                if overwatch_check != False:
                    move_bookings.append(overwatch_check)
                    actions[unit_id] = [unit.move(overwatch_check, repeat=0)]
                    continue

                # on ice, but not over cargo capacity limit. Dig ice
                elif on_ice and free_cargo_ice and not below_power_threshold_heavy:
                    actions[unit_id] = [unit.dig(repeat=0)]
                    #print(unit_id, "digging")
                
                # not on ice, not over cargo capacity limit. Move to ice
                elif not on_ice and free_cargo_ice and not below_power_threshold_heavy:
                    direction = direction_to(unit.pos, closest_ice_tile)
                    move_bookings.append(coord_from_direction(x=unit.pos[0], y=unit.pos[1], direction=direction))
                    actions[unit_id] = [unit.move(direction, repeat=0)]
                    #print(unit_id, "move dig")
                
                # on factory, below power threshold. Pickup power
                elif on_factory and below_power_threshold_heavy:
                    direction = direction_to(unit.pos, closest_factory_tile)
                    actions[unit_id] = [unit.pickup(4, recharge_need_heavy, repeat=0)]
                    #print(unit_id, "pickup power")
                
                # not on factory, below power threshold. Move to recharge
                elif not on_factory and (below_power_threshold_heavy and not below_power_move_heavy):
                    direction = direction_to(unit.pos, closest_factory_tile)
                    move_bookings.append(coord_from_direction(x=unit.pos[0], y=unit.pos[1], direction=direction))
                    actions[unit_id] = [unit.move(direction, repeat=0)]
                    #print(unit_id, "move recharge")

                # ajacent to factory, over cargo limit. Transfer ice
                elif ajacent_factory and not free_cargo_ice and not below_power_threshold_heavy:
                    direction = direction_to(unit.pos, closest_factory_tile)
                    actions[unit_id] = [unit.transfer(direction, 0, unit.cargo.ice, repeat=0)]
                    #print(unit_id, "transfer")

                # not ajacent to factory. Move to transfer
                elif not ajacent_factory and not free_cargo_ice and not below_power_threshold_heavy:
                    direction = direction_to(unit.pos, closest_factory_tile)
                    move_bookings.append(coord_from_direction(x=unit.pos[0], y=unit.pos[1], direction=direction))
                    actions[unit_id] = [unit.move(direction, repeat=0)]
                    #print(unit_id, "move transfer")
                
                # not on factory, below critical power. Do nothing (recharge)
                elif not on_factory and (below_power_threshold_heavy and below_power_move_heavy):
                    #print(unit_id, "doing nothing (recharging)")
                    continue


            ### Finds all ore tiles, sorts by proximity to light bot
            ### Light bot logic
            if isLight:
                # sorts low rubble tiles by proximity to closest factory tile to unit
                sorted_rubble_tile_vision = sorted(filter(lambda x: (game_state.board.rubble[x[0]][x[1]] != 0) and (game_state.board.ice[x[0]][x[1]] != 1) and (game_state.board.ore[x[0]][x[1]] != 1), rubble_tile_locations_low_rubble), key=lambda coord: (coord[0] - closest_factory_tile[0])**2 + (coord[1] - closest_factory_tile[1])**2)
                lowest_rubble__tile_visible = sorted_rubble_tile_vision[0]

                closest_ore_tiles = sorted(ore_tile_locations, key=lambda p: (p[0] - unit.pos[0])**2 + (p[1] - unit.pos[1])**2 )
                closest_rubble_tiles = sorted(rubble_tile_locations, key=lambda p: (p[0] - unit.pos[0])**2 + (p[1] - unit.pos[1])**2 )
                closest_ore_tile = closest_ore_tiles[0]
                closest_rubble_tile = closest_rubble_tiles[0]
                
                # decides if the bot should be assigned an ore location or find closest rubble
                on_ore_assignment = mine_assignments < light_ore_miners
                if on_ore_assignment and mine_assignments < len(closest_ore_tiles):
                    closest_ore_tile = closest_ore_tiles[mine_assignments]
                    mine_assignments += 1
                if not on_ore_assignment and rubble_assignments < len(closest_rubble_tiles):
                    rubble_assignments += 1

                charge_state = game_state.is_day() and unit.power < 150 and not on_factory
                below_power_threshold_light = unit.power <= 6
                power_for_dig = unit.power >= unit.dig_cost(game_state)
                recharge_need_light = math.floor(150 - unit.power)
                on_ore = all(unit.pos == closest_ore_tile)
                on_rubble = all(unit.pos == closest_rubble_tile)
                free_cargo_ore = unit.cargo.ore < cargo_limit_ore

                ### performs overwatch check to see if HEAVY bots on flanks
                overwatch_check = light_overwatch(
                    unit=unit,
                    unit_x=unit.pos[0],
                    unit_y=unit.pos[1],
                    game_state=game_state,
                    opponent=self.opp_player
                )

                if overwatch_check != False:
                    newDirection = check_tile_occupation(
                        game_state=game_state,
                        unit_x=unit.pos[0],
                        unit_y=unit.pos[1],
                        direction=overwatch_check,
                        booked_coords=move_bookings,
                        player=self.player,
                        opponent=self.opp_player
                    )
                    move_bookings.append(coord_from_direction(x=unit.pos[0], y=unit.pos[1], direction=newDirection))
                    actions[unit_id] = [unit.move(newDirection, repeat=0)]
                    continue

                # Print out bot info 
                #print(unit.unit_type, unit_id, "at:", unit.pos, "power and ore:", unit.power, unit.cargo.ore, "unit queue:", len(unit.action_queue))

                # TODO perform some sort of check to see if bot has hostile bot (light = attack, heavy = flee) on horizontal/vertical tiles

                # on ore, under cargo limit, not below power threshold, not charge state, not charge state. Dig ore
                if on_ore and free_cargo_ore and not below_power_threshold_light and not charge_state:
                    actions[unit_id] = [unit.dig(repeat=0)]
                    #print(unit_id, "dig ore")
                

                # on rubble, under cargo limit, not below power threshold, not charge state. Dig rubble
                elif on_rubble and free_cargo_ore and not below_power_threshold_light and not charge_state:
                    actions[unit_id] = [unit.dig(repeat=0)]
                    #print(unit_id, "dig rubble")
                

                # on ore assignment, not on ore, not on rubble, under cargo limit, not below power threshold, not charge state. Move to ore
                elif on_ore_assignment and not on_ore and not on_rubble and free_cargo_ore and not below_power_threshold_light and not charge_state:
                    direction = direction_to(unit.pos, closest_ore_tile)
                    newDirection = check_tile_occupation(game_state=game_state,
                        unit_x=unit.pos[0],
                        unit_y=unit.pos[1],
                        direction=direction,
                        booked_coords=move_bookings,
                        player=self.player,
                        opponent=self.opp_player
                    )
                
                    if newDirection == 0:
                        continue
                        #print(unit_id, "recharge instead of moving")
                        
                    else:
                        move_bookings.append(coord_from_direction(x=unit.pos[0], y=unit.pos[1], direction=newDirection))
                        actions[unit_id] = [unit.move(newDirection, repeat=0)]
                        #print(unit_id, "move to ore")
                    

                # not on ore assignment, not on ore, not on rubble, under cargo limit, not below power threshold, not charge state. Move to rubble
                elif not on_ore_assignment and not on_rubble and free_cargo_ore and not below_power_threshold_light and not charge_state:
                    direction = direction_to(unit.pos, lowest_rubble__tile_visible)
                    newDirection = check_tile_occupation(game_state=game_state,
                        unit_x=unit.pos[0],
                        unit_y=unit.pos[1],
                        direction=direction,
                        booked_coords=move_bookings,
                        player=self.player,
                        opponent=self.opp_player
                    )

                    if newDirection == 0:
                        continue
                        #print(unit_id, "recharge instead of moving")
                        
                    else:
                        move_bookings.append(coord_from_direction(x=unit.pos[0], y=unit.pos[1], direction=newDirection))
                        actions[unit_id] = [unit.move(newDirection, repeat=0)]
                        #print(unit_id, "move to rubble")


                # below power threshold or in a charge state. Do nothing (recharge)
                elif below_power_threshold_light or charge_state:
                    continue
                    #print(unit_id, "recharge")
                

                # ajacent to factory, at/over cargo limit, not below power threshold, not charge state. Transfer ore
                elif ajacent_factory and not free_cargo_ore and not below_power_threshold_light and not charge_state:
                    direction = direction_to(unit.pos, closest_factory_tile)
                    actions[unit_id] = [unit.transfer(direction, 1, unit.cargo.ore)]
                    #print(unit_id, "transfer cargo")
                

                # not ajacent to factory, at/over cargo limit, not below power threshold, not charge state. Move to transfer
                elif not ajacent_factory and not free_cargo_ore and not below_power_threshold_light and not charge_state:
                    direction = direction_to(unit.pos, closest_factory_tile)
                    move_bookings.append(coord_from_direction(x=unit.pos[0], y=unit.pos[1], direction=direction))
                    actions[unit_id] = [unit.move(direction, repeat=0)]
                    #print(unit_id, "move to transfer cargo")

        '''
        # NB DO NOT REMOVE
        # SERVES AS BACKUP FOR TESTING if lux_functions stops working
        if game_state.env_steps >= 998:
            player0_total_lichen = 0
            player1_total_lichen = 0
            game_state: GameState = obs_to_game_state(step, env.state.env_cfg, obs)
            player0_factories = game_state.factories['player_0']
            player1_factories = game_state.factories['player_1']

            for factory_id, factory in player0_factories.items():
                lichen_strain_tiles = np.argwhere(game_state.board.lichen_strains == factory.strain_id)
                for tile in lichen_strain_tiles:
                    player0_total_lichen += game_state.board.lichen[tile[0]][tile[1]]


            for factory_id, factory in player1_factories.items():
                lichen_strain_tiles = np.argwhere(game_state.board.lichen_strains == factory.strain_id)
                for tile in lichen_strain_tiles:
                    player1_total_lichen += game_state.board.lichen[tile[0]][tile[1]]


            print("turn:", game_state.env_steps)
            print('player_0 lichen:', player0_total_lichen)
            print('player_1 lichen:', player1_total_lichen)
        '''

        #print()




        

        return actions



In [26]:
# imports from support files with class and functions
#from Submission1.agent import Archimedes_Lever_s
import submission_6_agent
import default_agent
import lux_functions

# recreate our agents and run
player0 = env.agents[0]
player1 = env.agents[1]
agents = {
    player0: Archimedes_Lever(env.agents[0], env.state.env_cfg), 
    player1: submission_6_agent.Archimedes_Lever_s(env.agents[1], env.state.env_cfg)#default_agent.Default_Agent(env.agents[1], env.state.env_cfg)
}
img = env.render("rgb_array", width=48, height=48)
px.imshow(img).show()
lux_functions.interact(env=env, agents=agents, steps=1000, video=True)

spawn choices 105
[(array([15, 15]), 6.5625), (array([13, 17]), 9.1875), (array([15, 16]), 9.5), (array([15, 14]), 9.625), (array([14, 17]), 11.5625), (array([12, 17]), 12.0), (array([32, 36]), 13.3125), (array([10,  4]), 13.5), (array([9, 3]), 13.5625), (array([31, 36]), 13.6875), (array([30, 36]), 13.875), (array([8, 3]), 14.1875), (array([10,  5]), 14.1875), (array([13, 13]), 15.0), (array([14, 13]), 15.3125), (array([29, 36]), 15.4375), (array([12, 13]), 16.3125), (array([10,  6]), 16.4375), (array([28, 36]), 17.0), (array([7, 3]), 17.9375), (array([27, 36]), 18.875), (array([33, 35]), 18.875), (array([11, 14]), 22.5), (array([9, 7]), 23.5), (array([11, 16]), 24.0625), (array([11, 15]), 24.125), (array([26, 35]), 24.25), (array([8, 7]), 25.25), (array([33, 34]), 25.5625), (array([6, 4]), 25.875), (array([6, 5]), 27.3125), (array([26, 34]), 27.8125), (array([7, 7]), 28.5), (array([33, 33]), 28.5625), (array([26, 33]), 28.75), (array([6, 6]), 29.125), (array([27, 32]), 30.125), (arra

OpenCV: FFMPEG: tag 0x30395056/'VP90' is not supported with codec id 167 and format 'webm / WebM'


In [27]:
### AGENT TESTING
import submission_6_agent
import default_agent
import lux_functions

tests = 10
results = []
for t in range(tests):
    test_env = LuxAI_S2() # create the environment object
    test_obs = test_env.reset(seed=np.random.seed()) # resets an environment with a seed

    # recreate our agents and run
    player0 = test_env.agents[0]
    player1 = test_env.agents[1]
    agents = {
        player0: Archimedes_Lever(test_env.agents[0], test_env.state.env_cfg), 
        player1: submission_6_agent.Archimedes_Lever_s(test_env.agents[1], test_env.state.env_cfg)#default_agent.Default_Agent(test_env.agents[1], test_env.state.env_cfg)
    }
    # UNCOMMENT IF MAPS SHOULD BE VISUALIZED
    '''
    img = test_env.render("rgb_array", width=48, height=48)
    px.imshow(img).show()
    '''
    test_results = lux_functions.test_agents(env=test_env, agents=agents, steps=1000)
    results.append(test_results)
    del test_env
    del test_obs

for result in results:
    print(result)

spawn choices 152
[(array([39,  1]), 5.875), (array([ 1, 33]), 12.5), (array([36,  8]), 12.75), (array([ 1, 21]), 14.125), (array([38,  4]), 14.5), (array([35,  7]), 17.3125), (array([35,  6]), 17.375), (array([41, 41]), 18.875), (array([40, 41]), 18.9375), (array([ 1, 34]), 21.625), (array([37,  4]), 22.75), (array([39, 41]), 23.0625), (array([38, 41]), 23.5625), (array([14, 11]), 24.4375), (array([ 5, 31]), 24.5625), (array([ 4, 22]), 24.625), (array([ 4, 23]), 24.8125), (array([11, 10]), 24.8125), (array([15, 10]), 25.5), (array([ 2, 21]), 26.166666666666668), (array([ 1, 32]), 26.25), (array([13, 11]), 26.6875), (array([34,  7]), 27.375), (array([12, 11]), 27.5625), (array([36,  5]), 27.75), (array([ 5, 34]), 28.25), (array([39,  2]), 28.333333333333332), (array([ 4, 24]), 29.0), (array([43,  1]), 29.5), (array([36,  9]), 29.6875), (array([41, 45]), 29.75), (array([39,  3]), 30.3125), (array([ 3, 21]), 30.6875), (array([ 5, 32]), 30.75), (array([11,  9]), 30.9375), (array([ 1, 14])

KeyboardInterrupt: 

In [ ]:
#!luxai-s2 archimedes_main.py main.py -v 2 -s 101 -o replay.html
#!tar -czf submission.tar.gz *